In [1]:
import pennylane as qml
import numpy as np

# 2-qubit statevector simulator
dev = qml.device("default.qubit", wires=2, shots=None)


In [2]:
def feature_map(x):
    # ensure numpy array of length 2
    x = np.asarray(x, dtype=float)
    qml.RX(x[0], wires=0)
    qml.RX(x[1], wires=1)
    qml.CZ(wires=[0, 1])


In [3]:
@qml.qnode(dev)
def kernel_element(x1, x2):
    feature_map(x1)
    qml.adjoint(feature_map)(x2)
    return qml.probs(wires=[0, 1])  # return all probabilities


In [4]:
x1 = np.array([0.5, 1.2])
x2 = np.array([0.4, 1.0])

probs_11 = kernel_element(x1, x1)
probs_22 = kernel_element(x2, x2)
probs_12 = kernel_element(x1, x2)

k11 = probs_11[0]  # |00> probability
k22 = probs_22[0]
k12 = probs_12[0]

print("K(x1,x1) =", k11)
print("K(x2,x2) =", k22)
print("K(x1,x2) =", k12)

K(x1,x1) = 1.0
K(x2,x2) = 1.0000000000000004
K(x1,x2) = 0.9875602675802709


In [5]:
def quantum_kernel(XA, XB):
    XA = np.asarray(XA, dtype=float)
    XB = np.asarray(XB, dtype=float)
    K = np.zeros((len(XA), len(XB)))
    for i in range(len(XA)):
        for j in range(len(XB)):
            K[i, j] = kernel_element(XA[i], XB[j])
    return K
